# How to use code cells in this notebook

If a code cell starts with 
```python
# RUN
```
Run the cell by CTRL+Enter, or the Run button above.  

If a code cell starts with
```python
# USER INPUT
```
User input is needed before running the cell. Usually there will be a cell preceding this which gives an example for the values to be provided.

If a code cell starts with
```python
# OPTIONAL USER INPUT
```
User input is needed before running the cell. However, some defaults are provided, so make sure that either the settings will work for your run, or change them appropriately.

If a cell starts with

**Example cell**

These cells are not code cells but examples of user inputs from the test data analysis for the actual code cell that follows it, informing the user about the formatting etc.

**Important note on entering input:** When entering user input, please make sure you follow the formatting provided in the example cells. For example, when the parameter is text, make sure you have quotation marks around the parameters but when it is a number, do not enclose in quotes. If it is a list, then provide a list in brackets.

In [2]:
# RUN
# these commands import necessary functions for the rest of the program
import sys
sys.path.append("/opt/src")
import mip_functions as mip
import os
import subprocess
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.lines import Line2D
plt.rcParams['svg.fonttype'] = 'none'
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
wdir = "/opt/analysis/"
data_dir = "/opt/data/"

Classes reloading.
functions reloading


#### Example cell

What is your wrangler output data called? This is usually a file name that ends in run_(experiment_id)\_wrangled\_(date).txt.gz and is in the folder that you bound as /opt/data. What sample sheet did you use when you wrangled the data? What species did you use? This will likely be the (species) portion of a folder called (species)\_species\_resources. For example, Plasmodium falciparum in the tutorial dataset is in a folder called pf_species_resources and has been assigned the species name 'pf'

```python

# provide the MIPWrangler output file
# which must be located in the /opt/data directory within the container.
info_file = "run_test_run_wrangled_20221102.txt.gz" 

# sample sheet used by the wrangler run (must be located in /opt/data)
sample_sheet = "sample_list.tsv"

# species name associated with the species_resources folder
species = 'pf'

# No input below
info_file = data_dir + info_file
sample_sheet = data_dir + sample_sheet
pd.read_table(sample_sheet).groupby(["sample_set", "probe_set"]).first()
```

In [ ]:
# USER INPUT

# provide the MIPWrangler output file
# which must be located in the /opt/data directory within the container.
info_file = "" 

# sample sheet associated with the wrangler file
sample_sheet = ""

# No input below
info_file = [data_dir + info_file]
sample_sheet = [data_dir + sample_sheet]
pd.read_table(sample_sheet[0]).groupby(["sample_set", "probe_set"]).first()

#### Example cell
Which sample sets and probe sets would you like to analyze? These are listed in your sample sheet under the "sample_set" and "probe_set" columns, and given to Jupyter as a [sample_set, probe_set] pair. You can also see an example row by running the Jupyter cell above.

```python
sample_groups = [["JJJ", "DR1,VAR4"]]
```

If more than one combination is to be used, the input will be a list of lists, for example:
```python
sample_groups = [["sample_set_1", "probe_set_1"], ["sample_set_2", "probe_set_2"]]
```

There are two closely related names in this section: probe_set and probe_sets_used. probe_set is a column from the input sample sheet. If a sample was captured/sequenced with multiple probe sets at the same time, there might optionally be multiple comma delimited probe sets in this column (e.g. DR1,VAR4 if sequencing was performed on DR1 and VAR4 probe sets). probe_sets_used is a string of the probe sets you'd like to analyze here in this Jupyter notebook (e.g. DR1 if you only want to analyze DR1 probes, or DR1,VAR4 if you'd like to analyze both DR1 and VAR4 probes.

probe_sets_used = "DR1"

In [7]:
# USER INPUT
sample_groups = [[]]

#### Example cell
How many processors would you like to use? Enter an integer that is less than or equal to the number of available processors on the computer/compute node that you are using. More processors means faster run time but higher likelihood of CPU crashes if your machine doesn't have enough RAM to handle the job.

```python
# available cpu count
processorNumber = 12

## extra bwa options for haplotype alignment
# use "-a" for getting all alignments
# use "-L 500" to penalize soft clipping 
# use "-t" to set number of available processors
bwaExtra = ["-t", str(processorNumber)]
```

In [9]:
# OPTIONAL USER INPUT
# available cpu count
processorNumber = 12

## extra bwa options for haplotype alignment
# use "-a" for getting all alignments
# use "-L 500" to penalize soft clipping 
# use "-t" to set number of available processors
bwaExtra = ["-t", str(processorNumber)]

### Get/Set the analysis settings
The cell below will retrieve a template of default analysis settings to use. It will then modify these settings to match the variables you defined above, and save them to whatever folder you bound to the singularity container's /opt/analysis folder.

In [10]:
# RUN

# copy the template settings file
temp_settings_file = "/opt/resources/templates/analysis_settings_templates/settings.txt"
subprocess.call(["scp", temp_settings_file, "/opt/analysis/template_settings.txt"])

# extract the settings template
temp_settings = mip.get_analysis_settings("/opt/analysis/template_settings.txt")

# update bwa settings with the options set above
bwaOptions = temp_settings["bwaOptions"]
try:
    bwaOptions.extend(bwaExtra)
except AttributeError:
    bwaOptions = [bwaOptions]
    bwaOptions.extend(bwaExtra)

# Create a list from the probe_sets string
mipSetKey = probe_sets_used.split(",") + [""]

# create a dictionary for which settings should be updated
# using the user specified parameters.
update_keys = {"processorNumber": processorNumber,
               "bwaOptions": bwaOptions,
               "species": species,
               "mipSetKey" : mipSetKey}
# update the settings
for k, v in update_keys.items():
    temp_settings[k] = v
# create a settings file in the analysis directory.
settings_file = "settings.txt"
settings_path = os.path.join(wdir, settings_file)
mip.write_analysis_settings(temp_settings, settings_path)
settings = mip.get_analysis_settings(wdir + settings_file)
# create probe sets dictionary
try:
    mip.update_probe_sets("/opt/project_resources/mip_ids/mipsets.csv",
                         "/opt/project_resources/mip_ids/probe_sets.json")
except IOError:
    pass

### MIPWrangler output file processing
The operation below filters and renames some of the columns from the original file.

In [7]:
# RUN
mip.process_info_file(wdir,
                      settings_file, 
                      info_files,
                      sample_sheets,
                      settings["mipsterFile"],
                      sample_sets=sample_groups)

## Filter and map haplotype sequences
Align each haplotype sequence to the reference genome. Remove off target haplotypes. All haplotype mappings will be saved to the disk so off targets can be inspected if needed. 

Some filters can be applied to remove noise:
*  minHaplotypeBarcodes: minimum total UMI cut off across all samples.
*  minHaplotypeSamples: minimum number of samples a haplotype is observed in.
*  minHaplotypeSampleFraction: minimum fraction of samples a haplotype is observed in.  

It is probably safe to apply minimal count filters like at least 10 UMIs across samples and at least two samples. However, most data sets will be easily handled without these filters. So it may be better to not filter at this step unless the downstream operations are taking too much resources. However, filters can and should be applied after variant calls are made.

#### Example cell
```python
# filter haplotype sequences based on the number of total supporting UMIs
settings["minHaplotypeBarcodes"] = 1
# filter haplotype sequences based on the number of samples they were observed in
settings["minHaplotypeSamples"] = 1
# filter haplotype sequences based on the fraction of samples they were observed in
settings["minHaplotypeSampleFraction"] = 0.0001
```

In [8]:
# OPTIONAL USER INPUT
# filter haplotype sequences based on the number of total supporting UMIs
settings["minHaplotypeBarcodes"] = 1
# filter haplotype sequences based on the number of samples they were observed in
settings["minHaplotypeSamples"] = 1
# filter haplotype sequences based on the fraction of samples they were observed in
settings["minHaplotypeSampleFraction"] = 0.0001 

In [ ]:
#RUN
mip.map_haplotypes(settings)
mip.get_haplotype_counts(settings)

### Preview the mapping results
Plotting the probe coverage by samples is a good  way to see overall experiment perfomance. It shows if a probe has at least 1 barcode (or however many is specified below) for a given sample.  

Dark columns point to poor performing probes whereas dark rows indicate poor samples. Note that this excludes samples with no reads at all. Use "all_barcode_counts.csv" file if those are of interest as well.

Some parameters can be supplied to the plotting function as noted in the comments.

In [ ]:
# OPTIONAL USER INPUT

# coverage filter: anything below this number will be considered absent
barcode_threshold = 10
# font size for tick labels for x and y axis
tick_label_size=5
# font size for heat map color bar
cbar_label_size=5
# figure resolution
dpi=300
# present/absent colors
absent_color='black'
present_color='green'
# Save the plot in the analysis directory?
# If false, plots the graph here.
save=False
# How frequent the x and y-axis ticks should be
# every nth column will have  a tick
ytick_freq=None
xtick_freq=None
# rotation of xtick labels
xtick_rotation=90

In [ ]:
# OPTIONAL USER INPUT
barcode_counts = pd.read_csv(wdir + "barcode_counts.csv",
             header = [0,1], index_col = 0)
mip.plot_performance(barcode_counts,
                     barcode_threshold=barcode_threshold,
                     tick_label_size=tick_label_size,
                     cbar_label_size=cbar_label_size,
                     dpi=dpi,
                     absent_color=absent_color,
                     present_color=present_color,
                     save=save,
                     ytick_freq=ytick_freq,
                     xtick_freq=xtick_freq,
                     xtick_rotation=xtick_rotation)

### Look at summary stats 
There are summary statistics and meta data (if provided) we can use to determine if coverage is enough, whether further sequencing is necessary, and how to proceed if further sequencing will be needed.

In [ ]:
# RUN
sample_summary = pd.read_csv(wdir + "sample_summary.csv")
sample_summary.head()

### Plot total barcode count vs probe coverage
A scatter plot of total barcode count vs number of probes covered at a certain barcode count is a good way to see how the relationship between total coverage and probe coverage, which is useful in determining how to proceed to the next experiments or analyses.

In [ ]:
# RUN
f = sns.pairplot(data = sample_summary,
                x_vars = "Barcode Count",
                y_vars = "targets_with_10_barcodes",
                plot_kws={"s": 10})
f.fig.set_size_inches(5,5)
f.fig.set_dpi(150)
_ = plt.xticks(rotation=45)

## Repooling capture reactions for further sequencing.
### Factors to consider:
1. What do you we want to accomplish? In most cases, we would like to get enough coverage for a number of probes for each sample. For example, the test data contains **50 probes** in total. Let's say it is sufficient if we had a coverage of **10** or more for each probe for a sample. Then, we would not want to sequence any more of that sample. 
```python
target_coverage_count = 50
target_coverage_key='targets_with_10_barcodes'
```
Alternatively, we can set a goal of a fraction of total probes to reach a certain coverage rather than an absolute number of probes. For 95% of the maximum number of probes observed (47 in this case): 
```python
target_coverage_fraction = 0.95
target_coverage_key='targets_with_10_barcodes'
```
Although we set our goal to 47 probes, it is likely that some sample will never reach that number regardless of how much we sequence, if there is a deletion in the region, for example. So it makes sense to set a total coverage threshold after which we don't expect more data. Looking at the plot above, it seems like after 1000 barcode counts, we would reach our goal for most samples. 
```python
high_barcode_threshold = 10000
```
Another metric to use for determining if we want to sequence a sample more is the average read count per barcode count. This value indicates we have sequenced each unique molecular index in our sample so many times, so when the value is high, it is unlikely that we'd get more UMIs by sequencing the same library more. It makes more sense for a fresh MIP capture from these samples if more data is needed.
```python
barcode_coverage_threshold=10
```
Some samples perform very poorly for one reason or another. There are two options for these samples for repooling consideration: 1) Repool as much as we can for the next run, 2) Assuming there is a problem in the capture reaction, set up a new MIP capture reaction for these samples. It makes more sense to use option 1 if this is the first sequencing data using this library. Use option 2 if this library have been repooled at a higher volume already, but still producing poor data.
```python
barcode_count_threshold=100 # samples below total barcode count of this value is considered low coverage
low_coverage_action='Repool' # what to do for low coverage samples (Repool or Recapture)
```
Sometimes a handful of samples show uneven coverage of loci, i.e. they have very good coverage of a handful of loci but poor coverage in others, which may point to a problem with the sample or the experiment in general. These samples are determined by comparing the subset of samples that reached the goal we set (completed samples) and those that have not. We look at the number of barcodes per probe for _completed_ samples and get 25th percentile (or other percentile as set) and assume that if a sample on average has this many barcodes per target, it should have reached the set goal. For example, if on average _completed_ samples, i.e. samples that cover 47 probes at 10 barcodes or more, have 10000 total barcodes, they would have ~200 (10000/47) barcodes per target covered. And if an _incomplete_ sample has 5000 total barcodes and only 10 targets covered, this value would be 500 for that sample and it would be flagged as **uneven coverage** in repooling document.
```python
assesment_key='targets_with_1_barcodes' # coverage key to compare "complete" and "incomplete" samples
good_coverage_quantile=0.25 # percentile to set the threshold
```

#### Example cell
```python
high_barcode_threshold = 10000
target_coverage_count = None
target_coverage_fraction = 0.95
target_coverage_key = 'targets_with_10_barcodes'
barcode_coverage_threshold = 10
barcode_count_threshold = 100
low_coverage_action = 'Recapture'
assesment_key = 'targets_with_1_barcodes'
good_coverage_quantile = 0.25
```

In [13]:
# USER INPUT
high_barcode_threshold = 
low_coverage_action = 

In [13]:
# OPTIONAL USER INPUT
target_coverage_count = None
target_coverage_fraction = 0.95
target_coverage_key = 'targets_with_10_barcodes'
barcode_coverage_threshold = 10
barcode_count_threshold = 100
assesment_key = 'targets_with_1_barcodes'
good_coverage_quantile = 0.25

In [ ]:
# RUN
meta = pd.read_csv(wdir + "run_meta.csv")
data_summary = pd.merge(sample_summary, meta)
mip.repool(wdir, 
           data_summary, 
           high_barcode_threshold, 
           target_coverage_count=target_coverage_count, 
           target_coverage_fraction=target_coverage_fraction, 
           target_coverage_key=target_coverage_key,
           barcode_coverage_threshold=barcode_coverage_threshold,
           barcode_count_threshold=barcode_count_threshold, 
           low_coverage_action=low_coverage_action,
           assesment_key=assesment_key,
           good_coverage_quantile=good_coverage_quantile,
           output_file='repool.csv')

### Inspect the repool document
Library to completion field in the repool document has the value (volume) of how much from a sample should be pooled for re-sequencing. These values are only rough estimates and care should be taken to make sure there will be enough material to sequence.

In [ ]:
# RUN
pd.read_csv(wdir + "repool.csv").head()